# Filtering ACMA Register of Radiocommuncation Licences

The Register of Radiocommunication Licences (RRL) is the publicly available database of radiocommunication licences, location and frequency ranges. However, the vastness of the available data presents challenges to extracting useful data for any given project.

The code below will isolate data relevant to mid-band spectrum usage with a primary focus on Wireless Broadband Access (WBA). Typically, spectrum used for WBA is allocated in a spectrum licence which are multidimensional.

### Spectrum Licence Elements:
- Frequency range(s)
- Geographical location
- Client

The purpose of the code below is provide data in a format which is compatible with an interactive treemap using squarify or plotly packages.

## RRL Complications

The RRL source dataset can be downloaded from the ACMA website. In order to compile this notebook on a local machine, extract the contents of the RRL zip file into the project folder, naming convention should remain the same.

The data required for visualisation exists in two files: auth_spectrum_freq.csv and licence.csv. While licence.csv contains information regarding the licence itself, it does not contain geographic or spectrum usage information. The code uses licence.csv to filter out every licence that is not a spectrum licence. 

> NOTE: There are apparatus licences used by NBN Co. for WBA.

The auth_spec_freq.csv file is not compatible with licence.csv seeing as there are multiple licence number entires whereas licence.csv contains only one unique entry per licence number. As a result, the licence.csv dataset is being used a key data set to the frequency value dataset.

## Downloading RRL

In order to successfully compile this notebook, the RRL database needs to be in directory. Acquire the dataset from the following website:

https://www.acma.gov.au/radiocomms-licence-data

Be sure to download the spectra_data.zip file and extract the contents into this directory. The gitignore file should contain the required file names if verification is required. The RRL files are not included in repo.


## Identify Spectrum Licences:

In [12]:
import pandas as pd
import numpy as np

lic = pd.read_csv('licence.csv')

# Remove unnecessary columns
lic = lic.drop(columns=['SV_ID', 
    'SS_ID', 
    'DATE_ISSUED', 
    'DATE_OF_EFFECT', 
    'DATE_OF_EXPIRY', 
    'STATUS', 
    'STATUS_TEXT', 
    'AP_ID', 
    'SHIP_NAME', 
    'BSL_NO', 
    'AP_PRJ_IDENT'])

# Isolate spectrum licences and reset index
lic = lic.loc[lic['LICENCE_TYPE_NAME'] == 'Spectrum']
specLic = lic.reset_index(drop=True, inplace=False)

# Create a series of Spectrum Licence IDs
specLicID = specLic['LICENCE_NO']

# Change column type from object to 64 bit integer
specLic = specLic.astype({'LICENCE_NO':'int64'})

# Filter out non-mid-band licences
specLic = specLic[~(specLic['LICENCE_CATEGORY_NAME'] == 'AWL - FSS Only')]
specLic = specLic[~(specLic['LICENCE_CATEGORY_NAME'] == 'AWL - Standard')]
specLic = specLic[~(specLic['LICENCE_CATEGORY_NAME'] == '26 GHz Band')]
specLic = specLic[~(specLic['LICENCE_CATEGORY_NAME'] == '20 GHz Defence Band')]
specLic = specLic[~(specLic['LICENCE_CATEGORY_NAME'] == '30 GHz Defence Band')]
specLic = specLic.reset_index(drop=True, inplace=False)

# Save to file
specLic.to_csv('ACMA Datasets/spectrumLicences.csv', index=False)
specLic

,LICENCE_NO,CLIENT_NO,LICENCE_TYPE_NAME,LICENCE_CATEGORY_NAME
0,10143110,1136980.0,Spectrum,2 GHz Band
1,10143136,1103274.0,Spectrum,2 GHz Band
2,10143466,1104504.0,Spectrum,2 GHz Band
3,10143562,1103276.0,Spectrum,2 GHz Band
4,10231258,1103276.0,Spectrum,1800 MHz Band
...,...,...,...,...
61,9469870,1149289.0,Spectrum,2.5 GHz Band
62,9469871,1103275.0,Spectrum,2.5 GHz Band
63,9469878,1103275.0,Spectrum,2.5 GHz Band
64,9599659,1103275.0,Spectrum,2.3 GHz Band


## Identify Frequency Ranges and Locations:

In [13]:
freq = pd.read_csv('auth_spectrum_freq.csv')

# Remove irrelevant data
freq = freq.drop(columns=['AREA_CODE'])

# Convert Hz to MHz
#freq = freq.div(10000000, axis=['LW_FREQUENCY_START', 'LW_FREQUENCY_END', 'UP_FREQUENCY_START', 'UP_FREQUENCY_END'])
freq['LW_FREQUENCY_START'] = (freq['LW_FREQUENCY_START'] / 1000000).round(3)
freq['LW_FREQUENCY_END'] = (freq['LW_FREQUENCY_END'] / 1000000).round(3)
freq['UP_FREQUENCY_START'] = (freq['UP_FREQUENCY_START'] / 1000000).round(3)
freq['UP_FREQUENCY_END'] = (freq['UP_FREQUENCY_END'] / 1000000).round(3)

# Save to file
freq.to_csv('ACMA Datasets/frequencyList.csv', index='False')
freq

,LICENCE_NO,AREA_NAME,LW_FREQUENCY_START,LW_FREQUENCY_END,UP_FREQUENCY_START,UP_FREQUENCY_END
0,10006309,Regional South Australia,1710.0,1712.5,1805.0,1807.5
1,10143110,Perth,1960.0,1970.0,2150.0,2160.0
2,10143110,Sydney,1920.0,1935.0,2110.0,2125.0
3,10143110,Adelaide,1960.0,1970.0,2150.0,2160.0
4,10143110,Melbourne,1920.0,1935.0,2110.0,2125.0
...,...,...,...,...,...,...
1440,9599659,2.3 GHz Band,2302.0,2337.0,NaN,NaN
1441,9619844,Sydney,1740.0,1755.0,1835.0,1850.0
1442,9619844,Merge: Brisbane + Merge: Adelaide + Perth,1745.0,1755.0,1840.0,1850.0
1443,9619844,Melbourne,1740.0,1755.0,1835.0,1850.0


## Combine and Filter Datasets

In [14]:
# Isolate frequency and location data for spectrum licences
specFreq = freq[freq['LICENCE_NO'].isin(specLic['LICENCE_NO'])]
specFreq = specFreq.reset_index(drop=True, inplace=False)

# Add band designations from specLic data frame
specBand = specLic[['LICENCE_NO', 'LICENCE_CATEGORY_NAME']]
specFreq = pd.merge(specFreq, specBand, how='left', on='LICENCE_NO')

# Add bandwidth values
specFreq['LW_BW'] = specFreq['LW_FREQUENCY_END'] - specFreq['LW_FREQUENCY_START']
specFreq['UP_BW'] = specFreq['UP_FREQUENCY_END'] - specFreq['UP_FREQUENCY_START']
specFreq['USAGE'] = specFreq['LW_BW'] + specFreq['UP_BW']

# Move LW_BW column
names = ['LICENCE_NO', 
    'AREA_NAME', 
    'LW_FREQUENCY_START', 
    'LW_FREQUENCY_END', 
    'LW_BW', 
    'UP_FREQUENCY_START', 
    'UP_FREQUENCY_END', 
    'UP_BW', 
    'USAGE', 
    'LICENCE_CATEGORY_NAME']
    
specFreq = specFreq.reindex(columns=names)
specFreq = specFreq.reset_index(drop=True, inplace=False)

# Save to file
specFreq.to_csv('ACMA Datasets/spectrumLicencesFreq.csv', index=False)
specFreq

,LICENCE_NO,AREA_NAME,LW_FREQUENCY_START,LW_FREQUENCY_END,LW_BW,UP_FREQUENCY_START,UP_FREQUENCY_END,UP_BW,USAGE,LICENCE_CATEGORY_NAME
0,10143110,Perth,1960.0,1970.0,10.0,2150.0,2160.0,10.0,20.0,2 GHz Band
1,10143110,Sydney,1920.0,1935.0,15.0,2110.0,2125.0,15.0,30.0,2 GHz Band
2,10143110,Adelaide,1960.0,1970.0,10.0,2150.0,2160.0,10.0,20.0,2 GHz Band
3,10143110,Melbourne,1920.0,1935.0,15.0,2110.0,2125.0,15.0,30.0,2 GHz Band
4,10143110,Brisbane,1960.0,1970.0,10.0,2150.0,2160.0,10.0,20.0,2 GHz Band
...,...,...,...,...,...,...,...,...,...,...
294,9599659,2.3 GHz Band,2365.0,2400.0,35.0,NaN,NaN,NaN,NaN,2.3 GHz Band
295,9599659,2.3 GHz Band,2302.0,2337.0,35.0,NaN,NaN,NaN,NaN,2.3 GHz Band
296,9619844,Sydney,1740.0,1755.0,15.0,1835.0,1850.0,15.0,30.0,1800 MHz Band
297,9619844,Merge: Brisbane + Merge: Adelaide + Perth,1745.0,1755.0,10.0,1840.0,1850.0,10.0,20.0,1800 MHz Band


## Uplink/Downlink Separated Dataset

For the purpose of high resolution data visualisation, uplinks and downlinks will need to be separated to for the algorithmic calculation of spectrum utilisation. The Spectrum frequency dataset from above will split the columns on LW and UP boundary. All NaN entries will be filter out and the results will be concatenated and sorted according to licence number.

The resulting dataset will contain individual frequency allocations which will greatly assist in determining utilisation of spectrum. The dataset must also contain 100MHz graduations of spectrum so as graphically represent spectrum utilisation.

In [15]:
# Separate Uplink and apply standard column names
uplink = specFreq[['LICENCE_NO',
    'AREA_NAME', 
    'UP_FREQUENCY_START', 
    'UP_FREQUENCY_END', 
    'UP_BW', 
    'LICENCE_CATEGORY_NAME']]

uplink = uplink[uplink['UP_BW'].notna()]
uplink['TYPE'] = 'UPLINK'
uplink.rename(columns={'UP_FREQUENCY_START':'START','UP_FREQUENCY_END':'END', 'UP_BW':'BW'}, inplace=True)
uplink.reset_index(drop=True, inplace=True)

# Separate Downlink and apply standard column names
downlink = specFreq[['LICENCE_NO', 
    'AREA_NAME', 
    'LW_FREQUENCY_START', 
    'LW_FREQUENCY_END', 
    'LW_BW', 
    'LICENCE_CATEGORY_NAME']]
    
downlink = downlink[downlink['LW_BW'].notna()]
downlink['TYPE'] = 'DOWNLINK'
downlink.rename(columns={'LW_FREQUENCY_START':'START','LW_FREQUENCY_END':'END', 'LW_BW':'BW'}, inplace=True)
downlink.reset_index(drop=True, inplace=True)

# Concatenate Uplink and downlink dataframes and sort by LICENCE_NO and START
specUsage = pd.concat([uplink, downlink], axis=0)
specUsage.reset_index(drop=True, inplace=True)
specUsage.sort_values(by=['LICENCE_NO','START'], inplace=True)

# Introduce 100MHz granularity
specUsage['RANGE'] = (np.trunc((specUsage['START'] / 100))*100)
specUsage.to_csv('ACMA Datasets/spectrumUsage.csv', index=False)
specUsage


,LICENCE_NO,AREA_NAME,START,END,BW,LICENCE_CATEGORY_NAME,TYPE,RANGE
363,9263429,Sydney,825.0,835.0,10.0,800 MHz Band,DOWNLINK,800.0
364,9263429,Adelaide,825.0,835.0,10.0,800 MHz Band,DOWNLINK,800.0
365,9263429,Brisbane,825.0,835.0,10.0,800 MHz Band,DOWNLINK,800.0
366,9263429,Melbourne,825.0,835.0,10.0,800 MHz Band,DOWNLINK,800.0
367,9263429,Perth,825.0,835.0,10.0,800 MHz Band,DOWNLINK,800.0
...,...,...,...,...,...,...,...,...
360,11286131,Merge: ma044 + Merge: ma026 + ma043,3400.0,3475.0,75.0,3.4 GHz Band,DOWNLINK,3400.0
361,11345733,Area 1,2500.0,2510.0,10.0,2.5 GHz Band,DOWNLINK,2500.0
96,11345733,Area 1,2620.0,2630.0,10.0,2.5 GHz Band,UPLINK,2600.0
362,11345734,Area 1,2500.0,2510.0,10.0,2.5 GHz Band,DOWNLINK,2500.0


## Geographical Irrespective Dataset

Using the Uplink/Downlink separated dataset, to create a data set without AREA_NAMES and duplicate removed spectrum allocations. By removing AREA_NAMES, duplicate row entries would indicate geographic differences in spectrum allocations. Removing duplicates should result in a dataset reflecting spectrum usage in Australia overall. 

> NOTE: The following dataset will contain spectrum allocation overlaps as a result of ignoring geographical differences.

In [16]:
downlink = downlink[['LICENCE_NO',
    'START', 
    'END', 
    'BW', 
    'LICENCE_CATEGORY_NAME', 
    'TYPE']]
    
downlink.sort_values(by=['LICENCE_NO','START'], inplace=True)
downlink.drop_duplicates(inplace=True)
downlink.reset_index(drop=True, inplace=True)

uplink = uplink[['LICENCE_NO', 
    'START', 
    'END', 
    'BW', 
    'LICENCE_CATEGORY_NAME', 
    'TYPE']]
    
uplink.sort_values(by=['LICENCE_NO','START'], inplace=True)
uplink.drop_duplicates(inplace=True)
uplink.reset_index(drop=True, inplace=True)

nonGeoSpecific = pd.concat([uplink, downlink], axis=0)
nonGeoSpecific['RANGE'] = (np.trunc((nonGeoSpecific['START'] / 100))*100)
nonGeoSpecific

,LICENCE_NO,START,END,BW,LICENCE_CATEGORY_NAME,TYPE,RANGE
0,9263429,870.0,880.0,10.0,800 MHz Band,UPLINK,800.0
1,9263429,870.0,875.0,5.0,800 MHz Band,UPLINK,800.0
2,9263433,875.0,890.0,15.0,800 MHz Band,UPLINK,800.0
3,9263433,880.0,890.0,10.0,800 MHz Band,UPLINK,800.0
4,9263448,1850.0,1865.0,15.0,1800 MHz Band,UPLINK,1800.0
...,...,...,...,...,...,...,...
153,11286129,3510.0,3575.0,65.0,3.4 GHz Band,DOWNLINK,3500.0
154,11286129,3542.5,3575.0,32.5,3.4 GHz Band,DOWNLINK,3500.0
155,11286131,3400.0,3475.0,75.0,3.4 GHz Band,DOWNLINK,3400.0
156,11345733,2500.0,2510.0,10.0,2.5 GHz Band,DOWNLINK,2500.0


# DATA VISUALISATION:

## Display Heirarchical Data Using Existing Classifiers

Use the spectrumLicencesFreq dataset to create a tree map of the existing dataset. Note that this dataset is imperfect:
- Very long area designations (some of which are not well defined)
- Does not display the magnitude of allocation vs free spectrum
- Uses band categories as opposed to defined frequency graduations

In [17]:
import plotly.express as px

# First draft visualisation
fig = px.treemap(specFreq, path=[px.Constant('all'), 'LICENCE_CATEGORY_NAME','AREA_NAME','LICENCE_NO'], values='USAGE')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.write_html('ACMA Treemaps/treemap.html')
fig.show()

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



## Display Hierarchical Data with Created Classifiers

Using the spectrumUsage dataset, create a treemap that illustrates spectrum utilisation that distinguishes spectrum over a given area. This is proving impossible given the irregular area definitions.

The desired hierarchies include:
RANGE > AREA_NAME > START
AREA_NAME > RANGE > START
RANGE > START > LICENCE_NO

However, any hierarchy without AREA_NAME renders the graphic useless as spectrum usage will not be differentiated based on geographical location, resulting cells exceeding 100% usage of a given graduation.

Below are a few examples of various treemaps using AREA_NAME as part of their hierarchy. Note that all treemaps are not particularly useful in illustrating the distribution of frequency allocations due to obscure AREA_NAMES.

> NOTE: THE FOLLOWING TREE MAPS DO NOT DISPLAY SECTIONS OF UNALLOCATED SPECTRUM. CELL AREA CORRESPONDS TO FREQUENCY ALLOCATION BANDWIDTH WITH REFERENCE TO PARENT CELL.

In [18]:
# Spectrum Usage Data Set
fig = px.treemap(specUsage, path=[px.Constant('all'), 'RANGE','AREA_NAME','START','LICENCE_NO'], values='BW')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.write_html('ACMA Treemaps/usageTreemap.html')
fig.show()

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib

In [19]:
# Spectrum Usage Data Set
fig = px.treemap(specUsage, path=[px.Constant('all'), 'RANGE','START','LICENCE_NO'], values='BW')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.write_html('ACMA Treemaps/usageTreemap1.html')
fig.show()

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [20]:
# Spectrum Usage Data Set
fig = px.treemap(specUsage, path=[px.Constant('all'), 'AREA_NAME','RANGE','START'], values='BW')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.write_html('ACMA Treemaps/usageTreemap2.html')
fig.show()

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [21]:
# Spectrum Usage Data Set
fig = px.treemap(specUsage, path=[px.Constant('all'), 'LICENCE_CATEGORY_NAME','RANGE','START','LICENCE_NO'], values='BW')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.write_html('ACMA Treemaps/usageTreemap3.html')
fig.show()

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib

In [22]:
# Non Geographically Specific Representation
fig = px.treemap(nonGeoSpecific, path=[px.Constant('all'), 'LICENCE_CATEGORY_NAME','RANGE','START','LICENCE_NO'], values='BW')
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.write_html('ACMA Treemaps/nonGeoSpecificTreemap.html')
fig.show()

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib

## Data Visualisation Conclusions

Whilst plotly treemaps present opportunities for interactive data visualisation tools, the location/area classification need to be reviewed to better reflect useful information. Location/area classifications seem to be an independent variable which makes creating generalized treemaps challenging.

> It is possible to use the HCISEval tool and associated functions as well as area definitions (HCIS Codes) to create general area classifications (regions/states)

Potential to include geographical data using ArcMap but require access to hcis2kml.asp file (code the ACMA website uses to convert HCIS identifiers into XML files).